In [1]:
from netCDF4 import Dataset, num2date, date2num
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cartopy.crs as ccrs
import datetime as dt
import xarray as xr
import os.path
from os import path
import pyproj

# for suitable font size in plots
font = {'family' : 'serif', 
        #'weight' : 'bold', 
        'size'   : 19} 
plt.rc('font', **font)

# READ data

In [ ]:
"""USING XARRAY"""

# Start date of dataset
start_year = 2017
start_month = 1
start_day = 2
start_hour = 0

# End data of dataset
end_year = 2017
end_month = 1
end_day = 3
end_hour = 0

#analysis times
times = ['00', '06', '12','18']

#Number of days
num_days = dt.datetime(end_year,end_month,end_day, end_hour, 0,0) - \
dt.datetime(start_year,start_month,start_day, start_hour,0,0)

#number of analysis times
num_times = len(times)

#define longitude and latitude for data
lon=5.32415
lat= 60.39299

count=1

#loop through days
for i in range(0,num_days.days):
    YEAR = dt.datetime(start_year,start_month,start_day + i, 0, 0, 0).strftime('%Y')
    MONTH  = dt.datetime(start_year,start_month,start_day + i, 0, 0, 0).strftime('%m')
    DAY = dt.datetime(start_year,start_month,start_day + i, 0, 0, 0).strftime('%d')
    
    #loop through analysis times
    for j in range(num_times):
        #Open dataset
        URL = 'https://thredds.met.no/thredds/dodsC/meps25epsarchive/' + YEAR + '/' \
        + MONTH + '/' + DAY + '/' + 'meps_subset_2_5km_' + YEAR + MONTH + DAY + 'T' \
        + times[j] + 'Z.nc'
        ds = xr.open_dataset(URL)
        
        # Compute projected coordinates of lat/lon point
        proj = pyproj.Proj(ds.projection_lambert.proj4) 
        X,Y = proj(lon,lat)
        
        #Select only data from lat/lon point
        ds = ds.sel(x=X,y=Y,method='nearest')
        
        met_lat = ds.variables['latitude']
        met_lon = ds.variables['longitude']
        met_ser_time = ds.variables['time']
        met_forecast_ref_time = ds.variables['forecast_reference_time']
        
        
        if 'met_forecast_ref_time' not in locals():
            #start_point = 1
            #met_ser_time= met_ser_time_all(count-1) + int(times[j])/24
            print('WARNING: Data for', URL, ' does not exist. Fill with NaNs')
        else:
            #convert to DatetimeIndex and find location og the forecast ref time
            start_point = met_ser_time.to_index().get_loc(met_forecast_ref_time.values)
            met_ser_time = met_ser_time[start_point]
            print('Read data from:', URL)
            print(met_ser_time)
        
        
        """ ####### GETTING *ALL* THE VARIABLES ####### """
        # selecting data from one specific lon, lat location, and one specific ensamble member
        # ds.air_temperature_2m.sel(x=X,y=Y,method='nearest').isel(ensemble_member=1) to select ensamble member
        
        """NEAR SURFACE"""
        #ACCUMULATED PRECIPITATION AMOUNT
        precipitation_amount_acc = ds.precipitation_amount_acc
        
    
        #AIR PRESSURE AT SEA LEVEL
        air_pressure_at_sea_level = ds.air_pressure_at_sea_level
       
        #2M AIR TEMPERATURE
        air_temperature_2m = ds.air_temperature_2m
     
        
        #2M RH
        relative_humidity_2m = ds.relative_humidity_2m
        
        
        #AIR TEMP AT LOWEST LEVEL
        air_temperature_lowest_level = ds.air_temperature_lowest_level
        
        
        #10M X-WIND
        x_wind_10m = ds.x_wind_10m
        
        #10M Y-WIND
        y_wind_10m = ds.y_wind_10m
        
        # INTEGRAL OF SUFACE DOWNWARD SENSIBLE HEAT FLUX                                  
        integral_of_surface_dwn_SH = ds.integral_of_surface_downward_sensible_heat_flux_wrt_time
        
        
        # INTEGRAL OF SURFACE DOWNWELLING LONGWAVE FLUX                                
        integral_of_surface_downwelling_LW =  ds.integral_of_surface_downwelling_longwave_flux_in_air_wrt_time
       
        # INTEGRAL OF SURFACE DOWNWELLING SHORTWAVE FLUX                                               
        integral_of_surface_downwelling_SW = ds.integral_of_surface_downwelling_shortwave_flux_in_air_wrt_time 
        
        
        # INTEGRAL OF SURFACE NET DOWNWARD SHORTWAVE                                                  
        integral_of_surface_net_downward_SW = ds.integral_of_surface_net_downward_shortwave_flux_wrt_time
       
        
        # ATMOS BL THICKNESS
        atmosphere_boundary_layer_thickness = ds.atmosphere_boundary_layer_thickness
        
        
        """ATM 2D FIELDS"""                                               
        # HIGH TYPE COUD AREA FRACTION
        high_type_cloud_area_fraction  = ds.high_type_cloud_area_fraction
        
        # LOW TYPE CLOUD ARE FRACTION
        low_type_cloud_area_fraction  = ds.low_type_cloud_area_fraction
        
        # MEDIUM TYPE CLOUD AREA FRACTION
        medium_type_cloud_area_fraction = ds.medium_type_cloud_area_fraction 
    
        # CLOUD AREA FRACTION                                                  
        cloud_area_fraction = ds.cloud_area_fraction                                                   
        # FOG AREA FRACTION                                                  
        fog_area_fraction = ds.fog_area_fraction
           
            
                                             
        """ATM 3D FIELDS (925hPa, 850 hPa, 700 hPa, 500 hPa)"""
        """
        # AIR TEMPERATURE PRESSURE LEVELS
        air_temperature_pl = ds.air_temperature_pl.
        
        # GEOPOTENTIAL PRESSURE LEVEL                                                   
        #geopotential_pl =  ds.geopotential_pl.
        
        # RELATIVE HUMIDITY PRESSURE LEVELS                                                  
        relative_humidity_pl = ds.relative_humidity_pl.
        
        # X WIND PRESSURE LEVELS
        x_wind_pl = ds.x_wind_pl.
        
        # Y WIND PRESSURE LEVELS                                                  
        y_wind_pl = ds.y_wind_pl."""
        #if count == 1:    
        data = xr.merge([precipitation_amount_acc,air_pressure_at_sea_level,\
                            air_temperature_2m,relative_humidity_2m,\
                              air_temperature_lowest_level,x_wind_10m,\
                               y_wind_10m , integral_of_surface_dwn_SH,\
                                  integral_of_surface_downwelling_LW , integral_of_surface_downwelling_SW,\
                                  integral_of_surface_net_downward_SW , atmosphere_boundary_layer_thickness ,\
                                  high_type_cloud_area_fraction ,low_type_cloud_area_fraction ,\
                                  medium_type_cloud_area_fraction,cloud_area_fraction,\
                                  fog_area_fraction])
        """else:
            xr.concat(data.precipitation_amount_acc, ds.precipitation_amount_acc, 'model_runs')
            xr.concat(data.air_pressure_at_sea_level, ds.air_pressure_at_sea_level, dim='time')
            xr.concat(data.air_temperature_2m, ds.air_temperature_2m, dim='time')
            xr.concat(data.relative_humidity_2m, ds.relative_humidity_2m, dim='time')
            xr.concat(data.air_temperature_lowest_level, ds.air_temperature_lowest_level, dim='time')
            xr.concat(data.x_wind_10m, ds.x_wind_10m, dim='time')
            xr.concat(data.y_wind_10m, ds.y_wind_10m, dim='time')
            xr.concat(data.integral_of_surface_dwn_SH, ds.integral_of_surface_dwn_SH, dim='time')
            xr.concat(data.integral_of_surface_downwelling_LW, ds.integral_of_surface_downwelling_LW, dim='time')
            xr.concat(data.integral_of_surface_downwelling_SW, ds.integral_of_surface_downwelling_SW, dim='time')
            xr.concat(data.integral_of_surface_net_downward_SW, ds.integral_of_surface_net_downward_SW, dim='time')
            xr.concat(data.atmosphere_boundary_layer_thickness, ds.atmosphere_boundary_layer_thickness, dim='time')
            xr.concat(data.high_type_cloud_area_fraction, ds.high_type_cloud_area_fraction, dim='time')
            xr.concat(data.low_type_cloud_area_fraction, ds.low_type_cloud_area_fraction, dim='time')
            xr.concat(data.medium_type_cloud_area_fraction, ds.medium_type_cloud_area_fraction, dim='time')
            xr.concat(data.cloud_area_fraction, ds.cloud_area_fraction, dim='time')
            xr.concat(data.fog_area_fraction, ds.fog_area_fraction, dim='time')"""
        count = count + 1
        
    """ Save data to file"""
    #path ='C:\Users\Inger\OneDrive\Dokumenter\UiB\Master admin\Data\DATASET' 
    #data.to_netcdf('meps_subset_2_5km.nc')
    
        
            
                              
    

In [5]:
"""USING XARRAY #2"""

# Start date of dataset
start_year = 2017
start_month = 1
start_day = 2
start_hour = 0

# End data of dataset
end_year = 2017
end_month = 1
end_day = 3
end_hour = 0

#analysis times
times = ['00', '06', '12','18']

#Number of days
num_days = dt.datetime(end_year,end_month,end_day, end_hour, 0,0) - \
dt.datetime(start_year,start_month,start_day, start_hour,0,0)

#number of analysis times
num_times = len(times)

#define longitude and latitude for data
lon=5.32415
lat= 60.39299

count=0

#create empty data
data = {}

#loop through days
for i in range(0,num_days.days):
    YEAR = dt.datetime(start_year,start_month,start_day + i, 0, 0, 0).strftime('%Y')
    MONTH  = dt.datetime(start_year,start_month,start_day + i, 0, 0, 0).strftime('%m')
    DAY = dt.datetime(start_year,start_month,start_day + i, 0, 0, 0).strftime('%d')
    
    #loop through analysis times
    for j in range(num_times):
        URL = 'https://thredds.met.no/thredds/dodsC/meps25epsarchive/' + YEAR + '/' \
        + MONTH + '/' + DAY + '/' + 'meps_subset_2_5km_' + YEAR + MONTH + DAY + 'T' \
        + times[j] + 'Z.nc'
        
        
        ds = xr.open_dataset(URL)
        proj = pyproj.Proj(ds.projection_lambert.proj4)

        # Compute projected coordinates of lat/lon point
        X,Y = proj(lon,lat)
        
        met_lat = ds.variables['latitude']
        met_lon = ds.variables['longitude']
        met_ser_time = ds.variables['time']
        met_forecast_ref_time = ds.variables['forecast_reference_time']
        
        
        if 'met_forecast_ref_time' not in locals():
            start_point = 1
            met_ser_time= met_ser_time_all(count-1) + int(times[j])/24
            print('WARNING: Data for', URL, ' does not exist. Fill with NaNs')
        else:
            #convert to DatetimeIndex and find location og the forecast ref time
            start_point = met_ser_time.to_index().get_loc(met_forecast_ref_time.values)
            met_ser_time = met_ser_time[start_point]
            print('Read data from:', URL)
            
        ds = ds.sel(x=X,y=Y,method='nearest')
        
        
        
        """ ####### GETTING *ALL* THE VARIABLES ####### """
        # selecting data from one specific lon, lat location, and one specific ensamble member
        # ds.air_temperature_2m.isel(ensemble_member=1) to select ensamble member
        
         #ACCUMULATED PRECIPITATION AMOUNT
        precipitation_amount_acc = ds.precipitation_amount_acc
        
    
        #AIR PRESSURE AT SEA LEVEL
        air_pressure_at_sea_level = ds.air_pressure_at_sea_level
       
        #2M AIR TEMPERATURE
        air_temperature_2m = ds.air_temperature_2m
        
        if count == 0:
            data = xr.Dataset(data_vars=dict(met_forecast_ref_time = met_forecast_ref_time,\
                                                precipitation_amount_acc=precipitation_amount_acc,\
                                                air_pressure_at_sea_level=air_pressure_at_sea_level,\
                                                air_temperature_2m=air_temperature_2m),\
                                 coords = ds.coords,\
                                 attrs = ds.attrs ) 
            data = data.expand_dims('model_runs')
          
        else:
            newdata = xr.Dataset(data_vars=dict(met_forecast_ref_time = met_forecast_ref_time,\
                                                precipitation_amount_acc=precipitation_amount_acc,\
                                                air_pressure_at_sea_level=air_pressure_at_sea_level,\
                                                air_temperature_2m=air_temperature_2m),\
                                 coords = ds.coords,\
                                 attrs = ds.attrs )
            #xr.concat(data, dict(precipitation_amount_acc,air_pressure_at_sea_level, air_temperature_2m), 'model_runs')
            xr.concat([data, newdata], dim='model_runs')
        count = count +1
      

Read data from: https://thredds.met.no/thredds/dodsC/meps25epsarchive/2017/01/02/meps_subset_2_5km_20170102T00Z.nc
Read data from: https://thredds.met.no/thredds/dodsC/meps25epsarchive/2017/01/02/meps_subset_2_5km_20170102T06Z.nc
Read data from: https://thredds.met.no/thredds/dodsC/meps25epsarchive/2017/01/02/meps_subset_2_5km_20170102T12Z.nc
Read data from: https://thredds.met.no/thredds/dodsC/meps25epsarchive/2017/01/02/meps_subset_2_5km_20170102T18Z.nc


In [26]:
met_ser_time = ds.variables['time']
met_forecast_ref_time = ds.variables['forecast_reference_time']
#print(met_ser_time, met_forecast_ref_time)
start_point = met_ser_time.where(met_ser_time == met_forecast_ref_time)
print(start_point)
#met_ser_time = met_ser_time[start_point]

<xarray.Variable (time: 67)>
array(['2017-01-02T18:00:00.000000000',                           'NaT',
                                 'NaT',                           'NaT',
                                 'NaT',                           'NaT',
                                 'NaT',                           'NaT',
                                 'NaT',                           'NaT',
                                 'NaT',                           'NaT',
                                 'NaT',                           'NaT',
                                 'NaT',                           'NaT',
                                 'NaT',                           'NaT',
                                 'NaT',                           'NaT',
                                 'NaT',                           'NaT',
                                 'NaT',                           'NaT',
                                 'NaT',                           'NaT',
                      

In [8]:
xr.concat([data, newdata], dim='model_runs')
data


<xarray.Dataset>
Dimensions:                     (ensemble_member: 10, height0: 1, time: 67, height_above_msl: 1, height1: 1, pressure: 5, atmosphere_as_single_layer: 1, hybrid: 5, height3: 1, hybrid0: 1, model_runs: 1)
Coordinates: (12/14)
  * ensemble_member             (ensemble_member) int16 0 1 2 3 4 5 6 7 8 9
  * height0                     (height0) float32 0.0
  * time                        (time) datetime64[ns] 2017-01-02 ... 2017-01-...
    x                           float32 -5.299e+05
    y                           float32 -2.493e+05
    latitude                    float64 60.4
    ...                          ...
  * height1                     (height1) float32 2.0
  * pressure                    (pressure) float32 500.0 700.0 850.0 925.0 1e+03
  * atmosphere_as_single_layer  (atmosphere_as_single_layer) int16 0
  * hybrid                      (hybrid) float64 0.9862 0.9894 ... 0.9956 0.9985
  * height3                     (height3) float32 10.0
  * hybrid0                     (hybrid0) float64 0.9985
Dimensions without coordinates: model_runs
Data variables:
    precipitation_amount_acc    (model_runs, time, height0, ensemble_member) float32 ...
    air_pressure_at_sea_level   (model_runs, time, height_above_msl, ensemble_member) float32 ...
    air_temperature_2m          (model_runs, time, height1, ensemble_member) float32 ...
Attributes: (12/18)
    Conventions:                     CF-1.6
    institution:                     Norwegian Meteorological Institute, MET ...
    creator_url:                     met.no
    summary:                         MEPS (MetCoOp-Ensemble Prediction System...
    source:                          MEPS 2.5km
    min_time:                        2017-01-02 00:00:00Z
    ...                              ...
    license:                         http://met.no/sb5-met.no/Forskning/Infor...
    comment:                         none
    title:                           MEPS subset 2.5km
    NCO:                             "4.6.0"
    history:                         Wed Sep 12 08:53:54 2018 Adding forecast...
    DODS_EXTRA.Unlimited_Dimension:  time

<xarray.DataArray 'precipitation_amount_acc' (model_runs: 1, time: 67, height0: 1, ensemble_member: 10)>
array([[[[0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
          0.00000000e+00]],

        [[2.18009949e-03, 4.17995453e-03, 1.54666901e-02,
          3.61499786e-02, 4.34017181e-03, 7.50541687e-04,
          1.21709824e-01, 1.51014328e-02, 1.03092194e-03,
          2.14767456e-03]],

        [[2.47955322e-03, 4.68444824e-03, 5.48439026e-02,
          4.04987335e-02, 1.37882233e-02, 1.48010254e-03,
          1.32129669e-01, 1.72386169e-02, 1.44767761e-03,
          2.59971619e-03]],

        [[1.41448975e-02, 4.71115112e-03, 1.50051117e-01,
          4.04052734e-02, 1.41338348e-01, 4.18348312e-01,
          1.32333755e-01, 1.71966553e-02, 8.50786209e-01,
          4.76226807e-02]],

...

        [[2.40423889e+01, 2.31286621e+01,            nan,
                     nan,            nan,            nan,
                     nan,            nan,            nan,
                     nan]],

        [[2.40424957e+01, 2.31272964e+01,            nan,
                     nan,            nan,            nan,
                     nan,            nan,            nan,
                     nan]],

        [[2.40435486e+01, 2.31285172e+01,            nan,
                     nan,            nan,            nan,
                     nan,            nan,            nan,
                     nan]],

        [[2.40435638e+01, 2.31279144e+01,            nan,
                     nan,            nan,            nan,
                     nan,            nan,            nan,
                     nan]]]], dtype=float32)
Coordinates:
  * ensemble_member  (ensemble_member) int16 0 1 2 3 4 5 6 7 8 9
  * height0          (height0) float32 0.0
  * time             (time) datetime64[ns] 2017-01-02 ... 2017-01-04T18:00:00
    x                float32 -5.299e+05
    y                float32 -2.493e+05
    latitude         float64 60.4
    longitude        float64 5.325
Dimensions without coordinates: model_runs
Attributes:
    long_name:      Accumulated total precipitation
    standard_name:  precipitation_amount
    units:          kg/m^2
    grid_mapping:   projection_lambert
    _ChunkSizes:    [  1   1   1 949 739]

AttributeError: 'IndexVariable' object has no attribute 'get_loc'

In [23]:
met_ser_time.where(met_ser_time == met_forecast_ref_time)

<xarray.Variable (time: 67)>
array(['2017-01-02T18:00:00.000000000',                           'NaT',
                                 'NaT',                           'NaT',
                                 'NaT',                           'NaT',
                                 'NaT',                           'NaT',
                                 'NaT',                           'NaT',
                                 'NaT',                           'NaT',
                                 'NaT',                           'NaT',
                                 'NaT',                           'NaT',
                                 'NaT',                           'NaT',
                                 'NaT',                           'NaT',
                                 'NaT',                           'NaT',
                                 'NaT',                           'NaT',
                                 'NaT',                           'NaT',
                                 'NaT',                           'NaT',
                                 'NaT',                           'NaT',
                                 'NaT',                           'NaT',
                                 'NaT',                           'NaT',
                                 'NaT',                           'NaT',
                                 'NaT',                           'NaT',
                                 'NaT',                           'NaT',
                                 'NaT',                           'NaT',
                                 'NaT',                           'NaT',
                                 'NaT',                           'NaT',
                                 'NaT',                           'NaT',
                                 'NaT',                           'NaT',
                                 'NaT',                           'NaT',
                                 'NaT',                           'NaT',
                                 'NaT',                           'NaT',
                                 'NaT',                           'NaT',
                                 'NaT',                           'NaT',
                                 'NaT',                           'NaT',
                                 'NaT',                           'NaT',
                                 'NaT',                           'NaT',
                                 'NaT'], dtype='datetime64[ns]')
Attributes:
    long_name:      time
    standard_name:  time
    _ChunkSizes:    1